Star Battle is a grid-based logic puzzle. You are given a SxS square grid divided into S connected regions, and a number N. You must find the unique way to place N*S stars into the grid such that:

    Every row has exactly N stars.
    Every column has exactly N stars.
    Every region has exactly N stars.
    No two stars are horizontally, vertically, or diagonally adjacent.

Challenge

Write a program to solve a Star Battle puzzle in a reasonable amount of time. There's no strict time requirement, but you should run your program through to completion for at least one (N, S) = (2, 10) puzzle for it to count as a solution.

Feel free to use whatever input/output format is most convenient for you. In the examples below, first N is given, then the SxS grid is given, with each cell labeled by a letter corresponding to its region. The output is . for empty cells and * for cells containing a star. But you don't have to use this format.

Example input (N, S) = (1, 6)

    1
    AABBCC
    AABCCC
    AABCCC
    DDBBEE
    DDBBEF
    DDBBFF

In [10]:
a = '''1
AABBCC
AABCCC
AABCCC
DDBBEE
DDBBEF
DDBBFF'''

b = '''2
AAAABBCCCC
ADAABBBCBB
ADDBBBBBBB
DDDDBEEEEB
DDBBBBBBEB
FFFFGGHHHH
FIFFGGGHGG
FIIGGGGGGG
IIIIGJJJJG
IIGGGGGGJG'''

c = '''3
AAAAABBBBBCCCCC
ADDDDBBBBBEEECC
ADDDDDDBEEEEEEC
ADDDFDDBEEGEEEC
ADDFFFHHHGGGEEC
AAAFFFHHHGGGCCC
AAHHFHHIHHGHCCC
AAAHHHIIIHHHJJJ
AAAKKKIIIKKKKLJ
AAAMKKKIKKKMLLJ
NNNMMKKKKKMMLLJ
NNNOMMMMMMMLLLJ
NOOOOMMMMMOOLLL
NOOOOOMMMOOOLLL
NNOOOOOOOOOOLLL'''

from pulp import *
from collections import defaultdict

def solve_star_battle(inx):
    
    linx = inx.split('\n')
    n = int(linx[0])
    L = [(y,ix,iy) for ix,x in enumerate(linx[1:]) for iy,y in enumerate(x)]    
    s = len(linx[1])
    A = [[[] for y in range(s)] for x in range(s)]
    B = [['.' for y in x] for x in linx[1:]]
    
    LP_variables = []
    LP_constraints = []
    constraints = defaultdict(list)

    for x in L:
        
        regio,hori,vert = x[0],str(x[1]),str(x[2])
        st = ','.join([regio,hori,vert])
        v = LpVariable(st,cat='Binary')
        LP_variables += v
        
        
        A[x[1]][x[2]] = v
        constraints[regio].append(v)
        constraints['h' + hori].append(v)
        constraints['v' + vert].append(v)
    
    for x in constraints.values():
        Ae = LpAffineExpression([(y,1) for y in x])
        Lc = LpConstraint(Ae,sense=0,rhs=n)
        LP_constraints.append(Lc)
    
    for x in range(s-1):
        for y in range(s-1):
            Ae = LpAffineExpression([(A[x][y],1),
                                     (A[x+1][y],1),
                                     (A[x][y+1],1),
                                     (A[x+1][y+1],1)])
            Lc = LpConstraint(Ae,sense=-1,rhs=1)
            LP_constraints.append(Lc)
    
    prob = LpProblem()
    prob += LpAffineExpression([(x,1) for x in LP_variables])
    
    for x in LP_constraints:
        prob += x
    
    
    
    prob.solve(GLPK())    

    for v in prob.variables():
        if v.varValue:
            x,y = list(map(int,v.name[2:].split((','))))
            B[x][y] = '*'
    
    return '\n'.join(''.join(y for y in x) for x in B)

for x in [a,b,c]:
    print('\n',solve_star_battle(x))
    %timeit -n1 -r1 solve_star_battle(x)

PulpSolverError: PuLP: cannot execute glpsol.exe

In [ ]:
'''2
AAAABBCCCC
ADAABBBCBB
ADDBBBBBBB
DDDDBEEEEB
DDBBBBBBEB
FFFFGGHHHH
FIFFGGGHGG
FIIGGGGGGG
IIIIGJJJJG
IIGGGGGGJG'''